# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = "../WeatherPy/Output/cities.csv"
weather_data = pd.read_csv(weather_file)
weather_data.dropna(inplace = True)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Lat","Lng"]].astype(float)
Humidity = weather_data["Humidity"].astype(float)

In [5]:
fig = gmaps.figure(center = (15,-5),zoom_level =1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=Humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig                                 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
vacation_cities = weather_data.loc[(weather_data["Max Temp"] <= 78 ) 
                                   & (weather_data["Max Temp"] >= 68)
                                  & (weather_data["Wind Speed"] <= 10)
                                  & (weather_data["Cloudiness"] == 0)]     
vacation_cities = pd.DataFrame(vacation_cities[["City","Lat","Lng","Temperature","Country"]])
vacation_cities

,City,Lat,Lng,Temperature,Country
49,port alfred,-42.7826,147.0587,73.45,AU
91,sinnamary,-18.4750,-70.3042,71.60,CL
151,mao,-24.8716,25.8699,69.80,BW
160,mys shmidta,15.7432,-96.4661,70.63,MX
165,sola,-32.8833,-68.8000,73.40,AR
171,whitehorse,12.0708,1.7889,74.55,BF
180,krasnoselkup,-33.9180,25.5701,73.40,ZA
221,vanavara,20.1167,77.7167,74.03,IN
226,muros,5.5167,5.7500,77.95,NG
260,severo-kurilsk,14.8961,-24.4956,69.96,CV


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = vacation_cities
hotel_df["Hotel Name"] = ""

In [16]:
# params dictionary to update each iteration
vaca_params = {"radius": 50000,"types": "lodging","key": g_key}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_cities.iterrows():
    
    # get lat, lng from df - this is what isn't working
    lat = row["Lat"]
    lng = row["Lng"]    

    # change location each iteration while leaving original params in place
    vaca_params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params = vaca_params).json()
    
    results = name_address['results']

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print(f'Google has found: {results[0]["name"]} in {row[0]}')
    except:
        print("nope, not this one")
        pass

Google has found: St Ives Hobart Accommodation Tasmania in port alfred
Google has found: Hostal Jardin Del Sol in sinnamary
Google has found: Aquarian Tide Hotel in mao
Google has found: La Isla Huatulco in mys shmidta
Google has found: Hotel Mendoza in sola
Google has found: Auberge Les Accords in whitehorse
Google has found: Radisson Blu Hotel, Port Elizabeth in krasnoselkup
Google has found: Hotel Anuprabha & Lawns in vanavara
Google has found: BON Hotel Delta in muros
Google has found: Tortuga B&B in severo-kurilsk
Google has found: Lords Inn Somnath in esperance
Google has found: Píer Vitória Hotel - Hotel em Vitória/ES in saint-augustin
Google has found: Ogomono Doumbo in astoria
Google has found: Royal Park hotel & suites in yankton
Google has found: Hotel Praia Morena in vardo
Google has found: Sur Plaza Hotel in fallon
Google has found: Gir Jungle Lodge in zhanatas
Google has found: Moon light Furnished Units in grafton
Google has found: Sun City Restaurant in grand river sout

In [17]:
hotel_df

,City,Lat,Lng,Temperature,Country,Hotel Name
49,port alfred,-42.7826,147.0587,73.45,AU,St Ives Hobart Accommodation Tasmania
91,sinnamary,-18.4750,-70.3042,71.60,CL,Hostal Jardin Del Sol
151,mao,-24.8716,25.8699,69.80,BW,Aquarian Tide Hotel
160,mys shmidta,15.7432,-96.4661,70.63,MX,La Isla Huatulco
165,sola,-32.8833,-68.8000,73.40,AR,Hotel Mendoza
171,whitehorse,12.0708,1.7889,74.55,BF,Auberge Les Accords
180,krasnoselkup,-33.9180,25.5701,73.40,ZA,"Radisson Blu Hotel, Port Elizabeth"
221,vanavara,20.1167,77.7167,74.03,IN,Hotel Anuprabha & Lawns
226,muros,5.5167,5.7500,77.95,NG,BON Hotel Delta
260,severo-kurilsk,14.8961,-24.4956,69.96,CV,Tortuga B&B


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center = (15,-5),zoom_level =1.5)
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))